In [1]:
import requests
from pprint import pprint

end_point = 'https://apis.data.go.kr/1471000/HtfsInfoService03'
encoding = 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA%3D%3D'
decoding = 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA=='

url = 'http://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01?'

# 3) 요청 변수(params) 세팅
params = {
    "pageNo":    "1",          # 페이지 번호
    "numOfRows": "3",         # 한 페이지 결과 수
    "ServiceKey": 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA==', # Swagger에 표시된 정확한 이름(ServiceKey)
    "type":      "json",       # 응답 포맷(xml/json) – default: xml
    # (필요시 API별 추가 파라미터를 여기에 더합니다)
}

# 4) 실제 요청 보내기
response = requests.get(url, params=params, timeout=10)
response.raise_for_status()

# 5) 결과 확인
pprint(response.json())

{'body': {'items': [{'item': {'BASE_STANDARD': '1. 성상 : 고유의 향미가 있고 이미·이취가 없는 '
                                               '노랑 하양색의 입자성이 있는 분말\n'
                                               '2. 프로바이오틱스 수(표시량 '
                                               '300,000,000,000(3,000억) CFU/g) '
                                               ': 표시량 이상\n'
                                               '3. 대장균군 : 음성\n'
                                               '4. 납 : 1.0 mg/kg 이하\n'
                                               '5. 카드뮴 : 0.3 mg/kg 이하',
                              'DISTB_PD': '제조일로부터 24개월까지',
                              'ENTRPS': '일동바이오사이언스(주)',
                              'INTAKE_HINT1': '1. 질환이 있거나 의약품 복용 시 전문가와 '
                                              '상담하십시오.\n'
                                              '2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 '
                                              '수 있습니다.\n'
                                              '3. 어

# Task
Given a JSON object containing health functional food ingredient information, explain how to process this data and store it in a vector database for later use, potentially including searching.

## 데이터 추출 및 정제

### Subtask:
JSON 응답에서 필요한 정보(`item` 필드 내의 텍스트 데이터)를 추출하고, 불필요한 문자나 형식을 제거하여 텍스트 데이터를 정제합니다.


**Reasoning**:
Extract and clean the text data from the JSON response and store the cleaned text and STTEMNT_NO in separate lists.



In [3]:
import re

response_data = response.json()
items = response_data['body']['items']

cleaned_texts = []
sttement_nos = []

for item_entry in items:
    item_data = item_entry['item']
    combined_text = ""
    for key, value in item_data.items():
        if isinstance(value, str):
            combined_text += f"{key}: {value}\n"

    # Remove unnecessary characters and extra whitespace
    cleaned_text = re.sub(r'[\n\r]', ' ', combined_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    cleaned_texts.append(cleaned_text)
    sttement_nos.append(item_data.get('STTEMNT_NO'))

# Display the first few cleaned texts and STTEMNT_NOs
print("Cleaned Texts:")
pprint(cleaned_texts[:3])
print("\nSTTEMNT_NOs:")
pprint(sttement_nos[:3])

Cleaned Texts:
['ENTRPS: 일동바이오사이언스(주) PRDUCT: 11종 혼합유산균 STTEMNT_NO: 20140017002183 REGIST_DT: '
 '20201027 DISTB_PD: 제조일로부터 24개월까지 SUNGSANG: 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 '
 '있는 분말 SRV_USE: 건강기능식품 원료로 사용 PRSRV_PD: 냉장조건 INTAKE_HINT1: 1. 질환이 있거나 의약품 복용 '
 '시 전문가와 상담하십시오. 2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. 3. 어린이가 함부로 섭취하지 않도록 '
 '일일섭취량 방법을 지도해 주십시오. 4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. MAIN_FNCTN: 유산균 증식 및 '
 '유해균 억제·배변활동 원활에 도움을 줄 수 있음 BASE_STANDARD: 1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 '
 '하양색의 입자성이 있는 분말 2. 프로바이오틱스 수(표시량 300,000,000,000(3,000억) CFU/g) : 표시량 이상 3. '
 '대장균군 : 음성 4. 납 : 1.0 mg/kg 이하 5. 카드뮴 : 0.3 mg/kg 이하',
 'ENTRPS: 주식회사 성윤 에프엔지(F&G) PRDUCT: 6년근 고려홍삼정 PREMIUM STTEMNT_NO: 201100200015 '
 'REGIST_DT: 20150522 DISTB_PD: 제조일로부터 24개월 SUNGSANG: 암갈색의 액상으로 고유의 색택과 향미를 '
 '가지며 이미·이취가 없어야 한다. SRV_USE: 1일 3회, 1회 1스푼(3.2g)씩 물에 녹여 섭취 하십시요. PRSRV_PD: '
 '직사광선을 피해 건조하고 서늘한 곳에서 보관한다. INTAKE_HINT1: [홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 '
 '주의 2) 특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다. 3) 1

## 텍스트 분할 (청킹)

### Subtask:
너무 긴 텍스트는 의미를 잃지 않는 선에서 적절한 크기의 덩어리(chunk)로 나눕니다. 이는 임베딩 모델의 입력 크기 제한을 맞추고 검색 정확도를 높이는 데 도움이 됩니다.


**Reasoning**:
Define chunk size and overlap, then iterate through the cleaned texts to split them into chunks with associated statement numbers.



In [5]:
chunk_size = 256  # Define the size of each chunk
chunk_overlap = 32  # Define the overlap between consecutive chunks

chunked_data = []

for i, text in enumerate(cleaned_texts):
    sttement_no = sttement_nos[i]
    # Simple character-based splitting with overlap
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:min(end, len(text))]
        chunked_data.append({'chunk': chunk, 'STTEMNT_NO': sttement_no})
        if end >= len(text):
            break
        start += chunk_size - chunk_overlap

# Display the first few chunks
pprint(chunked_data[:5])

[{'STTEMNT_NO': '20140017002183',
  'chunk': 'ENTRPS: 일동바이오사이언스(주) PRDUCT: 11종 혼합유산균 STTEMNT_NO: 20140017002183 '
           'REGIST_DT: 20201027 DISTB_PD: 제조일로부터 24개월까지 SUNGSANG: 고유의 향미가 있고 '
           '이미·이취가 없는 노랑 하양색의 입자성이 있는 분말 SRV_USE: 건강기능식품 원료로 사용 PRSRV_PD: 냉장조건 '
           'INTAKE_HINT1: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 2. 알레르기 체질 '},
 {'STTEMNT_NO': '20140017002183',
  'chunk': '약품 복용 시 전문가와 상담하십시오. 2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. 3. '
           '어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오. 4. 이상사례 발생 시 섭취를 중단하고 전문가와 '
           '상담하십시오. MAIN_FNCTN: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음 '
           'BASE_STANDARD: 1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말 2. '
           '프로바이오틱'},
 {'STTEMNT_NO': '20140017002183',
  'chunk': ' 없는 노랑 하양색의 입자성이 있는 분말 2. 프로바이오틱스 수(표시량 300,000,000,000(3,000억) '
           'CFU/g) : 표시량 이상 3. 대장균군 : 음성 4. 납 : 1.0 mg/kg 이하 5. 카드뮴 : 0.3 '
           'mg/kg 이하'},
 {'STTEMNT_NO': '201100200015',
  'chunk': 'ENTRPS: 주식회사 성윤 에프엔지(F&G) PRDUCT: 

## 임베딩 생성

### Subtask:
정제되고 분할된 각 텍스트 청크에 대해 임베딩 모델(예: 문장 임베딩 모델)을 사용하여 벡터(수치형 배열)를 생성합니다.


**Reasoning**:
Generate vector embeddings for each text chunk using a sentence embedding model and store them along with the chunk and STTEMNT_NO.



In [7]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each chunk and store them
for entry in chunked_data:
    chunk_text = entry['chunk']
    embedding = model.encode(chunk_text)
    entry['embedding'] = embedding

# Display the first few entries with embeddings
for i in range(min(5, len(chunked_data))):
    print(f"STTEMNT_NO: {chunked_data[i]['STTEMNT_NO']}")
    print(f"Chunk: {chunked_data[i]['chunk']}")
    print(f"Embedding shape: {chunked_data[i]['embedding'].shape}")
    print("-" * 20)

STTEMNT_NO: 20140017002183
Chunk: ENTRPS: 일동바이오사이언스(주) PRDUCT: 11종 혼합유산균 STTEMNT_NO: 20140017002183 REGIST_DT: 20201027 DISTB_PD: 제조일로부터 24개월까지 SUNGSANG: 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말 SRV_USE: 건강기능식품 원료로 사용 PRSRV_PD: 냉장조건 INTAKE_HINT1: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 2. 알레르기 체질 
Embedding shape: (384,)
--------------------
STTEMNT_NO: 20140017002183
Chunk: 약품 복용 시 전문가와 상담하십시오. 2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. 3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오. 4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. MAIN_FNCTN: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음 BASE_STANDARD: 1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말 2. 프로바이오틱
Embedding shape: (384,)
--------------------
STTEMNT_NO: 20140017002183
Chunk:  없는 노랑 하양색의 입자성이 있는 분말 2. 프로바이오틱스 수(표시량 300,000,000,000(3,000억) CFU/g) : 표시량 이상 3. 대장균군 : 음성 4. 납 : 1.0 mg/kg 이하 5. 카드뮴 : 0.3 mg/kg 이하
Embedding shape: (384,)
--------------------
STTEMNT_NO: 201100200015
Chunk: ENTRPS: 주식회사 성윤 에프엔지(F&G) PRDUCT: 6년근 고려홍삼정 PREMIUM STTEMNT_NO: 201100200015 REGIS

## 벡터 데이터베이스 선택 및 설정

### Subtask:
사용할 벡터 데이터베이스(예: Chroma, FAISS, Pinecone, Weaviate 등)를 선택하고 설정합니다.


**Reasoning**:
Select and set up the vector database. ChromaDB is chosen for its ease of use. This involves installing the library, initializing the client, and creating a collection to store the embeddings and metadata.



In [9]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 590.4 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.6 MB/s eta

In [11]:
import chromadb

# 1. Select and install ChromaDB (installation is done here via pip)
# !pip install chromadb

# 3. Initialize the ChromaDB client
client = chromadb.Client()

# 4. Create a collection to store embeddings and metadata
collection_name = "health_food_ingredients"
collection = client.get_or_create_collection(name=collection_name)

print(f"ChromaDB client initialized and collection '{collection_name}' created or retrieved.")

ChromaDB client initialized and collection 'health_food_ingredients' created or retrieved.


## 데이터 로드

### Subtask:
생성된 텍스트 청크와 해당 벡터 임베딩을 벡터 데이터베이스에 저장합니다. 이때 원본 데이터의 식별자(예: `STTEMNT_NO`)와 함께 저장하여 나중에 검색된 벡터에 해당하는 원본 데이터를 찾을 수 있도록 합니다.


**Reasoning**:
Prepare the data from `chunked_data` into lists suitable for adding to the ChromaDB collection and then add the data to the collection.



In [13]:
# Prepare lists for ChromaDB
ids = []
embeddings = []
documents = []
metadatas = []

# Iterate through the chunked_data and populate the lists
for i, entry in enumerate(chunked_data):
    # Create a unique ID for each chunk
    unique_id = f"{entry['STTEMNT_NO']}_{i}"
    ids.append(unique_id)
    embeddings.append(entry['embedding'].tolist()) # Convert numpy array to list
    documents.append(entry['chunk'])
    metadatas.append({'STTEMNT_NO': entry['STTEMNT_NO']})

# Add the data to the ChromaDB collection
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

# Print a confirmation message
print(f"Added {len(ids)} items to the '{collection_name}' collection.")
print(f"Current item count in collection: {collection.count()}")

Added 9 items to the 'health_food_ingredients' collection.
Current item count in collection: 9


## 검색 기능 구현 (선택 사항)

### Subtask:
벡터 데이터베이스에서 유사한 벡터를 검색하는 기능을 구현합니다. 사용자의 쿼리를 벡터화하여 데이터베이스에서 가장 유사한 벡터를 찾아 관련 데이터를 검색할 수 있습니다.


**Reasoning**:
Implement the search functionality by defining a query, generating its embedding, and querying the ChromaDB collection.



In [16]:
# 1. Define a query string
query_string = "피로 개선에 도움이 되는 건강기능식품"

# 2. Generate an embedding for the query string
query_embedding = model.encode(query_string).tolist()

# 3. Search for similar embeddings in the ChromaDB collection
search_results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=['documents', 'metadatas']
)

# 4. Process and print the search results
print(f"Search results for query: '{query_string}'")
print("-" * 30)

if search_results and search_results.get('documents'):
    for i in range(len(search_results['documents'][0])):
        document = search_results['documents'][0][i]
        metadata = search_results['metadatas'][0][i]
        sttement_no = metadata.get('STTEMNT_NO', 'N/A')
        # ChromaDB query results do not directly include similarity scores in the default output
        # We can only display the retrieved document and its metadata
        print(f"Result {i+1}:")
        print(f"  STTEMNT_NO: {sttement_no}")
        print(f"  Chunk: {document}")
        print("-" * 10)
else:
    print("No results found.")

Search results for query: '피로 개선에 도움이 되는 건강기능식품'
------------------------------
Result 1:
  STTEMNT_NO: 20140017002183
  Chunk: 약품 복용 시 전문가와 상담하십시오. 2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다. 3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오. 4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. MAIN_FNCTN: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음 BASE_STANDARD: 1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말 2. 프로바이오틱
----------
Result 2:
  STTEMNT_NO: 201100200015
  Chunk: ENTRPS: 주식회사 성윤 에프엔지(F&G) PRDUCT: 6년근 고려홍삼정 PREMIUM STTEMNT_NO: 201100200015 REGIST_DT: 20150522 DISTB_PD: 제조일로부터 24개월 SUNGSANG: 암갈색의 액상으로 고유의 색택과 향미를 가지며 이미·이취가 없어야 한다. SRV_USE: 1일 3회, 1회 1스푼(3.2g)씩 물에 녹여 섭취 하십시요. PRSRV_PD: 직사광선을 피해 건조하고 서늘한 곳에서 보관한다. INT
----------
Result 3:
  STTEMNT_NO: 20040015100230
  Chunk: ENTRPS: (주)일화 PRDUCT: 6년근 홍삼 자신만만 홍삼스틱 STTEMNT_NO: 20040015100230 REGIST_DT: 20200813 DISTB_PD: 제조일부터 2 년 SUNGSANG: 고유의 색택과 향미를 가지며 이미와 이취가 없는 액상 SRV_USE: 1일 1회, 1회 1포를 섭취하십시오. PRSRV_PD: 습기와 직사광선을 피하여 실온에 보관하시고, 개봉 후에는 반드시 냉장 보관하십시오. INTAKE_H

## Summary:

### Data Analysis Key Findings

*   The process successfully extracted and cleaned text data from a JSON response containing health functional food ingredient information.
*   Text data from individual ingredient records was combined and prepared for further processing.
*   The cleaned text was effectively split into smaller chunks of 256 characters with an overlap of 32 characters.
*   Each text chunk was successfully associated with its original statement number (`STTEMNT_NO`).
*   Vector embeddings were generated for each text chunk using the `all-MiniLM-L6-v2` sentence transformer model, resulting in 384-dimensional vectors.
*   A ChromaDB instance was initialized, and a collection named "health\_food\_ingredients" was created or retrieved.
*   The text chunks, their corresponding embeddings, unique IDs (combining `STTEMNT_NO` and chunk index), and metadata (`STTEMNT_NO`) were successfully added to the ChromaDB collection.
*   A vector search functionality was implemented, demonstrating the ability to query the database with a natural language string and retrieve relevant document chunks and their associated `STTEMNT_NO` based on semantic similarity.

### Insights or Next Steps

*   The established process provides a robust foundation for building a semantic search application for health functional food ingredients, allowing users to find relevant information using natural language queries.
*   Consider evaluating different chunking strategies (e.g., sentence-based splitting) and embedding models to potentially improve search accuracy and relevance for specific use cases.
